# 自動機械学習 Automated Machine Learning による品質管理モデリング & モデル解釈 (リモート高速実行)

製造プロセスから採取されたセンサーデータと検査結果のデータを用いて、品質管理モデルを構築します。
- Python SDK のインポート
- Azure ML service Workspace への接続
- Experiment の作成
- データの準備
- 計算環境の準備
- 自動機械学習の事前設定
- モデル学習と結果の確認
- モデル解釈

## 1. 事前準備
### Python SDK のインポート
Azure Machine Learning service の Python SDKをインポートします

In [1]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

### Azure ML workspace との接続
Azure Machine Learning service との接続を行います。Azure に対する認証が必要です。

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

azureml	eastus	mlservice	eastus


### 実験名の設定

In [3]:
# choose a name for experiment
experiment_name = 'automl-classif-factoryQC-remote'
experiment=Experiment(ws, experiment_name)

### 学習データの準備

In [4]:
dataset = Dataset.get_by_name(ws, name='factory')
dataset.take(5).to_pandas_dataframe()

,ID,Quality,ProcessA-Pressure,ProcessA-Humidity,ProcessA-Vibration,ProcessB-Light,ProcessB-Skill,ProcessB-Temp,ProcessB-Rotation,ProcessC-Density,ProcessC-PH,ProcessC-skewness,ProcessC-Time
0,1,0,7.00,0.27,0.36,20.70,0.04,45.00,170.00,1.00,3.00,0.45,8.80
1,2,0,6.30,0.30,0.34,1.60,0.05,14.00,132.00,0.99,3.30,0.49,9.50
2,3,0,8.10,0.28,0.40,6.90,0.05,30.00,97.00,1.00,3.26,0.44,10.10
3,4,0,7.20,0.23,0.32,8.50,0.06,47.00,186.00,1.00,3.19,0.40,9.90
4,5,0,7.20,0.23,0.32,8.50,0.06,47.00,186.00,1.00,3.19,0.40,9.90


In [5]:
train_dataset, test_dataset = dataset.random_split(0.8, seed=1234)

In [6]:
X_train = train_dataset.drop_columns(columns=['Quality','ID'])
y_train = train_dataset.keep_columns(columns=['Quality'], validate=True)
X_test  = test_dataset.drop_columns(columns=['Quality','ID'])
y_test  = test_dataset.keep_columns(columns=['Quality'], validate=True)

### 計算環境 (Machine Learning Compute) の設定

In [7]:
# 予め cpucluster という名称の Machine Learning Compute を作成しておく
from azureml.core.compute import ComputeTarget
compute_target = ComputeTarget(ws, "cpucluster")

In [8]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = compute_target
conda_run_config.environment.docker.enabled = True

cd = CondaDependencies.create(conda_packages=['numpy','py-xgboost<=0.80', 'tensorflow>=1.10.0,<=1.12.0'])
conda_run_config.environment.python.conda_dependencies = cd

## 2. 自動機械学習 Automated Machine Learning
### 学習事前設定

In [9]:
automl_settings = {
    "iteration_timeout_minutes": 5,
    "iterations": 50,
    "n_cross_validations": 3,
    "primary_metric": 'AUC_weighted',
    "preprocess": True,
    "enable_tf" : True,
    "enable_voting_ensemble": False,
    "enable_stack_ensemble": False
}

automl_config = AutoMLConfig(task = 'classification',
                             X = X_train,
                             y = y_train,
                             run_configuration=conda_run_config,
                             max_concurrent_iterations = 10,
                             **automl_settings
                            )

### 実行と結果確認

In [10]:
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote compute: cpucluster
Parent Run ID: AutoML_edbf905f-2c3d-4968-a32d-75588681cdb9
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   StandardScalerWrapper SGD                      0:01:14       0.7801    0.7801
         4   StandardScalerWrapper ExtremeRandomTrees       0:02:28       0.8123    0.8123
         6   StandardScalerWrapper SG

WARNING - Retrying (Retry(total=2, connect=2, read=3, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1a408a8710>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)': /history/v1.0/subscriptions/9c0f91b8-eb2f-484c-979c-15848c098a6b/resourceGroups/mlservice/providers/Microsoft.MachineLearningServices/workspaces/azureml/experiments/automl-classif-factoryQC-remote/runs/AutoML_edbf905f-2c3d-4968-a32d-75588681cdb9
WARNING - Retrying (Retry(total=1, connect=1, read=3, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1a408a82b0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)': /history/v1.0/subscriptions/9c0f91b8-eb2f-484c-979c-15848c098a6b/resourceGroups/mlservice/providers/Microsoft.MachineLearningServices/workspaces/azureml/exper

In [11]:
# Widget で結果確認
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 'sd…

In [12]:
# 詳細ログの出力
remote_run.get_details()

{'runId': 'AutoML_edbf905f-2c3d-4968-a32d-75588681cdb9',
 'target': 'cpucluster',
 'status': 'Completed',
 'startTimeUtc': '2019-10-03T06:38:55.846671Z',
 'endTimeUtc': '2019-10-03T06:57:40.468268Z',
 'properties': {'num_iterations': '50',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'MaxTimeSeconds': '300',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'cpucluster',
  'RawAMLSettingsString': "{'name': 'automl-classif-factoryQC-remote', 'path': None, 'subscription_id': '9c0f91b8-eb2f-484c-979c-15848c098a6b', 'resource_group': 'mlservice', 'workspace_name': 'azureml', 'region': 'eastus', 'compute_target': 'cpucluster', 'spark_service': None, 'iterations': 50, 'primary_metric': 'AUC_weighted', 'task_type': 'classification', 'data_script': None, 'validation_size': 0.0, 'n_cross_validations': 3, 'y_min': None, 'y_max': None, 'num_classes': None, 'preprocess': True, 'lag_length': 0, '

In [13]:
best_run, fitted_model = remote_run.get_output()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-classif-factoryQC-remote,AutoML_edbf905f-2c3d-4968-a32d-75588681cdb9_25,azureml.scriptrun,Completed,Link to Azure Portal,Link to Documentation


### モデルの理解

In [14]:
fitted_model.named_steps['datatransformer'].get_featurization_summary()

[{'RawFeatureName': 'ProcessA-Pressure',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'ProcessA-Humidity',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'ProcessA-Vibration',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'ProcessB-Light',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'ProcessB-Skill',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'ProcessB-Temp',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Tranformations': ['MeanImputer']},
 {'RawFeatureName': 'ProcessB-Rotation',
  'TypeDetected': 'N

In [15]:
from pprint import pprint


def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()


print_model(fitted_model)

datatransformer
{'enable_feature_sweeping': None,
 'feature_sweeping_timeout': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None}

StandardScalerWrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing.data',
 'with_mean': False,
 'with_std': False}

XGBoostClassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bytree': 0.5,
 'eta': 0.1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 9,
 'max_leaves': 0,
 'min_child_weight': 1,
 'missing': nan,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'reg:logistic',
 'random_state': 0,
 'reg_alpha': 0.9375,
 'reg_lambda': 0.5208333333333334,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': True,
 'subsample': 0.7,
 'tree_method': 'auto',
 'verbose': -10}



## 3. モデルの解釈

In [16]:
from azureml.train.automl.automl_explain_utilities import AutoMLExplainerSetupClass, automl_setup_model_explanations

automl_explainer_setup_obj = automl_setup_model_explanations(fitted_model, X=X_train, X_test=X_test, y=y_train, task='classification')

Current status: Setting up data for AutoMl explanations
Current status: Setting up the AutoML featurization for explanations
Current status: Setting up the AutoML estimator
Current status: Setting up the AutoML featurizer
Current status: Generating a feature map for raw feature importance
Current status: Finding all classes from the dataset
Current status: Data for AutoMl explanations successfully setup


In [17]:
from azureml.explain.model.mimic.models.lightgbm_model import LGBMExplainableModel
from azureml.explain.model.mimic_wrapper import MimicWrapper
explainer = MimicWrapper(ws, automl_explainer_setup_obj.automl_estimator, LGBMExplainableModel, 
                         init_dataset=automl_explainer_setup_obj.X_transform, run=best_run,
                         features=automl_explainer_setup_obj.engineered_feature_names, 
                         feature_maps=[automl_explainer_setup_obj.feature_map],
                         classes=automl_explainer_setup_obj.classes)

Using older than supported version of lightgbm, please upgrade to version greater than 2.2.1


In [18]:
raw_explanations = explainer.explain(['local', 'global'], get_raw=True, 
                                     raw_feature_names=automl_explainer_setup_obj.raw_feature_names,
                                     eval_dataset=automl_explainer_setup_obj.X_test_transform)
#print(raw_explanations.get_feature_importance_dict())

In [19]:
from azureml.contrib.explain.model.visualize import ExplanationDashboard
ExplanationDashboard(raw_explanations, automl_explainer_setup_obj.automl_pipeline, automl_explainer_setup_obj.X_test_raw)

ExplanationWidget(value={'predictedY': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0…